<a href="https://colab.research.google.com/github/aditya-malte/SemEval/blob/master/SemEval_BERT_Pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json

In [2]:
!git clone https://github.com/aditya-malte/bert-temporary-fork.git

Cloning into 'bert-temporary-fork'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 290.24 KiB | 1.07 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [3]:
%cd bert-temporary-fork/

/content/bert-temporary-fork


##Generate vocab.txt

In [4]:
!git clone https://github.com/kwonmha/bert-vocab-builder.git

Cloning into 'bert-vocab-builder'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (91/91), done.


In [5]:
!python /content/bert-temporary-fork/bert-vocab-builder/subword_builder.py \
--corpus_filepattern sample_text.txt \
--output_filename vocab.txt \
--min_count 5



W0124 14:00:13.851541 140272139405184 module_wrapper.py:139] From /content/bert-temporary-fork/bert-vocab-builder/tokenizer.py:133: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

['./sample_text.txt']

W0124 14:00:13.852832 140272139405184 module_wrapper.py:139] From /content/bert-temporary-fork/bert-vocab-builder/tokenizer.py:138: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

0.0017452239990234375 for reading read file : ./sample_text.txt
read all files

W0124 14:00:13.858881 140272139405184 module_wrapper.py:139] From /content/bert-temporary-fork/bert-vocab-builder/text_encoder.py:588: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Iteration 0
I0124 14:00:13.859014 140272139405184 text_encoder.py:588] Iteration 0
INFO:tensorflow:vocab_size = 376
I0124 14:00:13.874012 140272139405184 text_encoder.py:660] vocab_size = 376
INFO:tensorflow:Iteration 1
I0124 14:00:13.87413

##Generate pre-training corpus

In [0]:
with open('/content/bert-temporary-fork/vocab.txt') as f:
   vocab_size = sum(1 for _ in f)

In [7]:
print(vocab_size)

316


In [8]:
!python create_pretraining_data.py \
  --input_file=./sample_text.txt \
  --output_file=/tmp/tf_examples.tfrecord \
  --vocab_file=/content/bert-temporary-fork/vocab.txt \
  --do_lower_case=True \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5



W0124 14:00:18.632472 139879779469184 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0124 14:00:18.632648 139879779469184 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0124 14:00:18.632769 139879779469184 module_wrapper.py:139] From /content/bert-temporary-fork/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0124 14:00:18.633832 139879779469184 module_wrapper.py:139] From create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0124 14:00:18.634979 139879779469184 module_wrapper.py:139] From create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:*** Reading from input files ***
I012

##Set Hyperparameters

In [18]:
config_dict = {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size":2,
  "vocab_size": 28996
}
print(config_dict["vocab_size"])

28996


In [0]:
with open('bert_config.json', 'w') as fp:
    json.dump(config_dict, fp)

##Pre-train model

In [0]:
!python run_pretraining.py \
  --input_file=/tmp/tf_examples.tfrecord \
  --output_dir=/tmp/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=/content/bert-temporary-fork/bert_config.json \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20 \
  --num_warmup_steps=10 \
  --learning_rate=2e-5




W0124 14:02:59.202901 139870547756928 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0124 14:02:59.203087 139870547756928 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0124 14:02:59.203245 139870547756928 module_wrapper.py:139] From /content/bert-temporary-fork/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0124 14:02:59.203822 139870547756928 module_wrapper.py:139] From run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0124 14:02:59.203999 139870547756928 module_wrapper.py:139] From run_pretraining.py:418: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0124 14:02:59.204633 139870547756928 module_wrapper.py:139] From run_pretraining.py:420: T